In [2]:
import pandas as pd

region = pd.read_csv('C:/Users/franz/Desktop/ATCO/cfe-web-scraping/data/region.csv', encoding = 'ISO-8859-1')

In [3]:
region.head()

,id_region,estado,municipio,division
0,37,AGUASCALIENTES,AGUASCALIENTES,BAJÍO
1,41,AGUASCALIENTES,ASIENTOS,BAJÍO
2,39,AGUASCALIENTES,CALVILLO,BAJÍO
3,43,AGUASCALIENTES,COSIO,BAJÍO
4,44,AGUASCALIENTES,EL LLANO,BAJÍO


In [5]:
infra_data = pd.read_csv('C:/Users/franz/Desktop/ATCO/cfe-web-scraping/data/infraestructura_2021.csv')

infra_data.head()

,id_infraestructura,id_region,mes,distribucion,capacidad
0,1,37,ENERO,100.61,320.96
1,2,38,ENERO,100.61,320.96
2,3,39,ENERO,100.61,320.96
3,4,40,ENERO,100.61,320.96
4,5,41,ENERO,100.61,320.96


In [6]:
tarifas_data = pd.read_csv('C:/Users/franz/Desktop/ATCO/cfe-web-scraping/data/tarifas_2021.csv')

tarifas_data.head()

,id_tarifa,id_region,mes,base,intermedia,punta
0,1,37,ENERO,0.8927,1.5632,1.777
1,2,38,ENERO,0.8927,1.5632,1.777
2,3,39,ENERO,0.8927,1.5632,1.777
3,4,40,ENERO,0.8927,1.5632,1.777
4,5,41,ENERO,0.8927,1.5632,1.777


In [7]:
merged_data = pd.merge(tarifas_data, infra_data, on = 'id_region', how = 'inner')

In [8]:
merged_data = pd.merge(merged_data, region, on = 'id_region', how = 'inner')

In [9]:
grouped_data = merged_data.groupby(['estado', 'division']).agg({
    'base': ['mean', 'var'],
    'intermedia': ['mean', 'var'],
    'punta': ['mean', 'var'],
    'distribucion': ['mean', 'var'],
    'capacidad': ['mean', 'var']
}).reset_index()

In [11]:
grouped_data.columns = ['estado', 'division', 'base_mean', 'base_var', 'intermedia_mean', 'intermedia_var',
                        'punta_mean', 'punta_var', 'distribucion_mean', 'distribucion_var', 'capacidad_mean', 'capacidad_var']

# Mostrar los resultados
pd.set_option('display.max_columns', None)

display(grouped_data)

,estado,division,base_mean,base_var,intermedia_mean,intermedia_var,punta_mean,punta_var,distribucion_mean,distribucion_var,capacidad_mean,capacidad_var
0,AGUASCALIENTES,BAJÍO,0.896400,0.000007,1.570433,0.000028,1.785367,0.000038,97.561212,300.478578,312.920606,3092.646949
1,BAJA CALIFORNIA,BAJA CALIFORNIA,0.531600,0.000002,0.810200,0.000006,0.000000,0.000000,92.360000,0.000000,294.506667,1.288659
2,BAJA CALIFORNIA SUR,BAJA CALIFORNIA SUR,1.636493,0.195673,2.070933,0.313356,0.000000,0.000000,86.202667,542.841702,197.109333,2838.793779
3,CAMPECHE,PENINSULAR,1.043767,0.000011,1.738200,0.000035,1.936367,0.000045,93.190000,0.000000,322.646667,1.535753
4,CAMPECHE,SURESTE,0.860100,0.000007,1.472500,0.000027,1.648133,0.000035,145.350000,0.000000,283.803333,1.321825
...,...,...,...,...,...,...,...,...,...,...,...,...
88,VERACRUZ,SURESTE,0.858850,0.000007,1.470050,0.000024,1.645350,0.000031,145.350000,0.000000,283.265000,1.190700
89,YUCATÁN,PENINSULAR,1.042150,0.000008,1.735250,0.000027,1.933050,0.000034,93.190000,0.000000,322.035000,1.158357
90,ZACATECAS,BAJÍO,0.885332,0.008373,1.550716,0.025693,1.762913,0.033207,99.550947,105.800924,318.509368,1084.165274
91,ZACATECAS,BAJÍO Y GOLFO CENTRO,0.894508,0.000006,1.566738,0.000022,1.781115,0.000029,100.610000,0.000000,321.786923,1.185256


### Muestreo aleatorio para municipios.

Para cada combinación de estado y división, tomamos una muestra aleatoria de municipios. 

Para validar si la muestra es representativa, comparamos la varianza de la muestra con la varianza de toda la población.

Si la diferencia entre la varianza de la muestra y la de la población es pequeña, entonces es una muestra representativa. Supongamos que $dif < 0.05%$

In [10]:
import numpy as np

# Función para generar el muestreo aleatorio de municipios para un estado y división

def sample_municipios(df, estado, division, sample_size):

    subset = df[(df['estado'] == estado) & (df['division'] == division)]

    sample = subset.sample(n = sample_size, random_state = 42)

    return sample

# Cálculo de la varianza de la muestra vs varianza de la población

def compare_variance(df, sample, cols):

    pop_var = df[cols].var()

    sample_var = sample[cols].var()

    diff = np.abs(pop_var - sample_var)

    return pop_var, sample_var, diff

estado = 'AGUASCALIENTES'

division = 'BAJÍO'

sample_size = 3

sample = sample_municipios(merged_data, estado, division, sample_size)

columns = ['base', 'intermedia', 'punta', 'distribucion', 'capacidad']

population_var, sample_var, diff_var = compare_variance(
    merged_data[(merged_data['estado'] == estado) & (merged_data['division'] == division)],
    sample,
    columns
)

# Mostrar los resultados
print("Varianza de la población completa:")
print(population_var)
print("\nVarianza de la muestra:")
print(sample_var)
print("\nDiferencia absoluta entre la varianza de la población y la muestra:")
print(diff_var)

Varianza de la población completa:
base             0.000422
intermedia       0.001609
punta            0.002148
distribucion     0.000000
capacidad       87.616655
dtype: float64

Varianza de la muestra:
base              0.000346
intermedia        0.001321
punta             0.001765
distribucion      0.000000
capacidad       169.834033
dtype: float64

Diferencia absoluta entre la varianza de la población y la muestra:
base             0.000076
intermedia       0.000288
punta            0.000383
distribucion     0.000000
capacidad       82.217379
dtype: float64
